In [7]:
pip install mlxtend --upgrade --no-deps

In [8]:
pip install openpyxl

In [9]:
import numpy as np
import pandas as pd

# library untuk visualisasi data
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

# libarary untuk algoritma apriori
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth

In [11]:
# membaca dataset
triwulan1 = pd.read_excel('1hari.xlsx', 'triwulan1')
frames = [triwulan1]
df = pd.concat(frames)
df

,TANGGAL,NO TRANSAKSI,NAMA BARANG
0,2020-01-01,1JA100,FLORIDINA 320ML
1,NaT,NaN,HELLO SINGKONG BALADO PEDAS
2,NaT,NaN,TWISTKO JGG BKAR 70GR
3,NaT,NaN,NQJ KAOS KAKI JEMPOL
4,NaT,NaN,TOOP RASA GRUBI UNGU
...,...,...,...
1641,NaT,NaN,POTA BEE 68GR BBQ
1642,NaT,NaN,KRUPUK TENGIRI IKAN KEMBAR
1643,NaT,NaN,NUTRIBOOST 240ML COKLAT BTL
1644,NaT,NaN,TISSUE TODAY 2PLAY 250S


In [12]:
# Mengisi data NaN dengan data yang diatasnya
df.fillna(method='ffill', inplace=True)

# menghapus baris sub total pada kolom nama barang
df = df[~df["NAMA BARANG"].str.contains("SUB TOTAL")]

df

,TANGGAL,NO TRANSAKSI,NAMA BARANG
0,2020-01-01,1JA100,FLORIDINA 320ML
1,2020-01-01,1JA100,HELLO SINGKONG BALADO PEDAS
2,2020-01-01,1JA100,TWISTKO JGG BKAR 70GR
3,2020-01-01,1JA100,NQJ KAOS KAKI JEMPOL
4,2020-01-01,1JA100,TOOP RASA GRUBI UNGU
...,...,...,...
1640,2020-01-01,1JA9,SISIR MOTIF CEWEK
1641,2020-01-01,1JA9,POTA BEE 68GR BBQ
1642,2020-01-01,1JA9,KRUPUK TENGIRI IKAN KEMBAR
1643,2020-01-01,1JA9,NUTRIBOOST 240ML COKLAT BTL


In [13]:
products = df['NAMA BARANG'].unique()
print("Total produk: {}".format(len(products)))
# transaction = df['NO TRANSAKSI'].unique()
# print("Total transaksi: {}".format(len(transaction)))

Total produk: 539


In [14]:
one_hot = pd.get_dummies(df['NAMA BARANG'])
df.drop('NAMA BARANG', inplace=True, axis=1)
df = df.join(one_hot)
df

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,TANGGAL,NO TRANSAKSI,AB TALI RAMBUT GELANG,ABC 250ML JUICE ASAM JAWA,ABC 250ML LECI,ABC 250ML MANGGA,ABC EXO 230ML CHOCOMALT C,ABC EXO 230ML MILK COFFE,ABRID CARANG UNGU ISI12,ABRID KRIPIK BUAH SALAK/NANGKA,ABRID SALE GULUNG,ABRID SALE PISANG GORENG,ABRID TAHU PONG 160GR,ABRID TAHU UNYIL,ADEMSARI CHINGKU 320ML KLG,ADEMSARI CHINGKU 350ML BTL,ADES 600ML,AIM LOVER COKLAT 80GR,AL MANDURY 150GR,AL MANDURY 300GR,AL MANDURY AMPLANG 100GR,AL MANDURY MAKARONI NYOR2 200G,AL MANDURY WIDARAN,ALAMO 1500ML,ALAMO 600ML,AMANDA KRIPIK PISANG,AMANDA MAKARONI NYONYOR,ANTANGIN MINT,AQUA 1500ML,AQUA 450ML CLICK N GO,AQUA 600ML,ASTRI KRIPIK JAMUR TIRAM,ASTRI KRIPIK USUS BALADO/ORI,BALLPOIN FASTER HG EC3,BAYGON 600ML FLOWER GARDEN,BBIORE 100ML M BW SPARKLIN,BBIORE 250ML BTL LOVELY CS,BEAR BRAND 140ML WHITE TEA,BENG BENG 22GR,BENG BENG MAXX 32GR,...,TOOP RASA SAGON,TOOP RASA SALE GORENG,TOOP RASA SARANG MADU,TOOP RASA TAHU UNYIL,TOOP RASA TELA-TELA,TOP 16GR STROBERI,TOP 38GR XL TRIPLE CHOC,TORA CAFE ICED MILKY L 180ML,TRS 10GR SC BRIGHTENER /PINK,TRS 10GR ULTIMATE FLEK MLM,TRS 10GR ULTIMATE GLOW MLM,TWISTER THINS 20GR STROBERI,TWISTKO JGG BAKAR 145GR,TWISTKO JGG BKAR 70GR,TYPE X KIRIKO,ULTRA MILK 200ML COKLAT,ULTRA MILK 200ML LOWFAT PLN,ULTRA MILK 200ML MOKA,ULTRA MILK 250ML COKLAT,ULTRA MILK 250ML MOKA,ULTRA MIMI 125ML VAN,VEGETA HERBAL ANGGUR/SCHT,"VEGETA JERUK 6,3GR",VFRESH 10ML BARLEY MINT,VFRESH 10ML ORIGINAL,VIT 1500ML,VIT 600ML,VIT GELAS,VIT LEVITE 250ML BERRIES,WALLS CRNTO CLASIC STRAW&VNL,WALLS FEAST COKLAT,WALLS FEAST DOUBLE CHOC,WALLS FEAST VANILA,WALLS POPULAIRE STROBERI,WD 15ML BB CRM LIGHTENG LIGHT,WD REF LIGHT TWC (02),YAKULT SLOP,YOU C 1000 VIT ORANGE 140ML,ZINC 80ML BLACKSHINE,ZINC 80ML REFRESHING COOL
0,2020-01-01,1JA100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2020-01-01,1JA100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2020-01-01,1JA100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2020-01-01,1JA100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2020-01-01,1JA100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1640,2020-01-01,1JA9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1641,2020-01-01,1JA9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1642,2020-01-01,1JA9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1643,2020-01-01,1JA9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
# transaksi = (df.groupby(['NO TRANSAKSI', 'NAMA BARANG'])['NAMA BARANG']
#           .sum().unstack().reset_index()
#           .set_index('NO TRANSAKSI'))

In [16]:
transaksi = df.groupby(["NO TRANSAKSI","TANGGAL"])[products[:]].apply(sum)
transaksi = transaksi.reset_index()[products]
transaksi

,FLORIDINA 320ML,HELLO SINGKONG BALADO PEDAS,TWISTKO JGG BKAR 70GR,NQJ KAOS KAKI JEMPOL,TOOP RASA GRUBI UNGU,SILVERQUEEN 30GR GREEN TEA,NBT HANSEL 34GR,MIE CUP SEDAAP SOTO 77GR,NUVO FML 76GR ENERGIZING,CUSSON KIDS SMPO 100ML STBRI,H&S 75ML LEMON FRESH,NUVO FML 76GR CARE PROTECT,ROKOK SAMPOERNA MILD 16,CANDI MAS KURNIA RAMBAK SAPI,ROKOK DJARUM SUPER 12,POTA BEE 68GR BBQ,HATARI MALKIST C COCONUT 120G,TORA CAFE ICED MILKY L 180ML,AQUA 1500ML,ROKOK LA ICE 16,BISKUAT COKLAT 67.2GR,FRISIAN F 225ML PUREF COKLAT,FRESTEA 500ML APPLE,ROMA SARI GANDUM CKLT 115GR,LAYS 68GR RUMPUT LAUT,FRISIAN F 180ML BTL STROBERI,MALANG SINGKONG MADURA,SMAX RING KEJU 50GR,TARO 10GR SEAWEED,ROKOK LA BOLD 20,AMANDA KRIPIK PISANG,CRUNCH CHIPS 30GR CHOCOLATE,KOPIKO BLISTER CLASSIC 24GR,JAPOTA UMAMI JPNSE SEAWED 68GR,AL MANDURY 300GR,AQUA 600ML,DK SUKRO 70GR,TOOP RASA KRIPIK PISANG,DK SUKRO 140GR,FRESTEA 350ML HONEY,...,FATIGON SPIRIT,HEMAVITON ACTION/BIJI,DETTOL 105GR SKINCARE HK,ROKOK LA MENTHOL 16,SG ORAL B SHN C SOFT,SG FORMULA SP TRENDY S,ZINC 80ML BLACKSHINE,LASEGAR BTL 200ML,SG CIPTADENT CRYSTAL CLEAN M,SOKLIN SMART 800GR WHITE,TOOP RASA TELA-TELA,EKONOMI EV 500 K 184GR,DK SUKRO 140GR BBQ,POP MIE GORENG SPC 80GR,PERMEN MENTOS MINT,PERMEN TAMARIN 150GR,NESCAFE 200ML WHITE COFFEE,ROKOK SURYA PROFESIONAL,GARNIER M 100ML PW BRIGHFOAM,TOOP RASA CEKER AYAM,KAPAL API SPECIAL MIX 25GR,INDOMILK UHT 190ML COKLAT,ABC 250ML MANGGA,PERMEN FOXS 125GR FRUITY MINTS,SARI ROTI SANDWICH COKLAT,ROKOK DJARUM MLD BLACK S 16,TWISTER THINS 20GR STROBERI,SARI ROTI SANDWICH KEJU,ROKOK GG SHIVER MENTHOL 16,MUSTIKA WANGI 25ML KECIL,BHIMA SUN FLWR SEEDS 150G,CLUB 600ML,FRESTEA 500ML HONEY,ROKOK DJARUM BLACK 16,BIORE PORE PACK BLACK,CIPTADENT 75GR WHITENING,IMPLORA URBAN LIP MATTE 03,FRISIAN F 180ML MILKY COKLAT,FRISIAN F 250ML L.FAT CKLT,SISIR MOTIF CEWEK
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
413,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
414,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
415,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
transaksi.to_excel('data_penjualan.xlsx')